# This noteboook focuses on How to return a value from component and feed it to another

In [1]:
from kfp import dsl
from kfp.components import create_component_from_func
import kfp
from typing import NamedTuple

# Function creation
- Here return_number function is created that return a integer.

    Then print_something recieves the value and prints it

In [2]:
def return_number() -> NamedTuple('Output', [('number', int)]):
    return ([5])

def print_something(data: int):
    print(data+10)

# Pipeline Design
- Here create_component_from_func is used to convert function into components. The output from return_number_op is then fed into print_something_op

In [3]:
@dsl.pipeline(
    name='Return Number',
    description='A pipeline that retuns and adds number'
)
def pipeline():
    return_number_op = create_component_from_func(func=return_number)
    print_something_op = create_component_from_func(func=print_something)
    
    return_number_task = return_number_op()
    print_something_op(return_number_task.outputs['number'])
    

# Pipeline run
- Here pipeline experiment name, function is defined first, then using the pipeline funcion and yaml filename the pipeline is compiled that generated the .yaml file. Then the pipeline is uploaded using the client and it is executed to run at the end.

In [49]:
from datetime import datetime
import uuid
EXPERIMENT_NAME = "Print_Something"
pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + f'{uuid.uuid1()}.pipeline.yaml'
kfp.compiler.Compiler().compile(pipeline_func, pipeline_filename)
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = pipeline_func.__name__ + str(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
client.upload_pipeline(pipeline_filename)
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename)